In [1]:
import sys
sys.path.append('../src/')
import decode_mcd
import decode_mcd.multi_objective_problem as MOP
from decode_mcd import data_package


from decode_mcd import design_targets
from decode_mcd import counterfactuals_generator

In [33]:
pd.DataFrame([[0.5, "positive"]], columns = ["P1", "P2"])

,P1,P2
0,0.5,positive


In [42]:
import random

from pymoo.core.variable import Real, Choice

import numpy as np
import pandas as pd

x = pd.DataFrame(np.random.random(100).reshape(100, 1), columns = ["P1"])
x["P2"] = np.random.choice(["positive", "negative"], 100)

def calculate_score(_x):
    mask = _x["P2"] == "positive"
    res = _x["P1"] * (2*mask.astype(int)-1) #returns P1 if P2 is "positive" and -P1 if P2 is "negative"
    return np.expand_dims(res.values, axis=1)
    
y = calculate_score(x)

data = data_package.DataPackage(features_dataset=x,
                           predictions_dataset=y,
                           query_x= pd.DataFrame([[0.5, "positive"]], columns = ["P1", "P2"]),
                           design_targets=design_targets.DesignTargets([design_targets.ContinuousTarget(label=0,
                                                                          lower_bound=-0.01,
                                                                          upper_bound=0.1)]),
                           datatypes=[Real(bounds=(0, 1)), Choice(options=["negative", "positive"])])

problem = MOP.MultiObjectiveProblem(data_package=data,
                                prediction_function=lambda design: calculate_score(design),
                                constraint_functions=[])

generator = counterfactuals_generator.CounterfactualsGenerator(problem=problem,
                                     pop_size=10,
                                     initialize_from_dataset=False)

generator.generate(n_generations=3)
counterfactuals = generator.sample_with_dtai(num_samples=10, gower_weight=1,
                                             avg_gower_weight=1, cfc_weight=1,
                                             diversity_weight=50, include_dataset=False)
print(counterfactuals)

Initial population randomly initialized!
Training GA from 0 to 3 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        9 |      1 |  0.000000E+00 |  0.4799739633 |             - |             -
     2 |       19 |      2 |  0.000000E+00 |  0.2682488370 |  1.0000000000 |         ideal
     3 |       29 |      2 |  0.000000E+00 |  0.1624852085 |  0.000000E+00 |             f
Only found 2 valid counterfactuals! Returning all 2.
Done! Returning CFs
         P1        P2
0  0.080235  positive
1  0.098807  positive
